In [1]:
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in c:\programdata\anaconda3\lib\site-packages (0.0)


In [2]:
import pandas as pd
from datetime import datetime as dt
import numpy as np
import requests
import config
import json
import time
import datetime
from pprint import pprint

In [3]:
df = pd.read_csv('data_files/fullplayerstatslist.csv')

In [4]:
df

,Unnamed: 0,Unnamed: 0.1,First_Name,Last_Name,player_id,Position,Points,Free_Throw_Percent,Two_Pt_Percent,Three_Pt_Percent,...,Turnovers,Team,Home_Away,win,Team_points,Min_played,Crowd,Stadium_Cap,game_id,game_date
0,0,0,LeBron,James,0afbe608-940a-4d5d-a1f7-468718c67d91,F,19,50.0,81.818,0.0,...,4,Cavaliers,1,1,117,32:23,20562,20562,0da78f13-73ac-4465-8e31-ecc3029a5dc6,2016-10-25T23:30:00+00:00
1,1,1,James,Jones,09d25155-c3be-4246-a986-55921a1b5e61,G-F,5,100.0,0.000,100.0,...,0,Cavaliers,1,1,117,5:30,20562,20562,0da78f13-73ac-4465-8e31-ecc3029a5dc6,2016-10-25T23:30:00+00:00
2,2,2,J.R.,Smith,5934134d-0d27-42ea-a554-4b0e3e85ce56,G-F,8,0.0,20.000,25.0,...,0,Cavaliers,1,1,117,25:14,20562,20562,0da78f13-73ac-4465-8e31-ecc3029a5dc6,2016-10-25T23:30:00+00:00
3,3,3,Kay,Felder,8d3acdd5-9b5a-4d69-9912-de42d979c31a,G,0,0.0,0.000,0.0,...,0,Cavaliers,1,1,117,00:00,20562,20562,0da78f13-73ac-4465-8e31-ecc3029a5dc6,2016-10-25T23:30:00+00:00
4,4,4,Mike,Dunleavy,4ec1bff7-ec1b-488b-8a24-aed83e62b4ce,G-F,4,0.0,100.000,0.0,...,0,Cavaliers,1,1,117,22:32,20562,20562,0da78f13-73ac-4465-8e31-ecc3029a5dc6,2016-10-25T23:30:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182364,8965,8965,Marial,Shayok,0a530034-a07c-4db3-a99e-17c5254c07e1,G-F,5,100.0,0.000,33.3,...,1,76ers,0,1,134,7:56,Covid,8000,0fca55cb-4bc8-4646-a2e6-75d95a20270b,2020-08-15T01:00:00+00:00
182365,8966,8966,Josh,Richardson,ef11cca9-6605-44e8-943e-193c7b821465,G,5,0.0,50.000,50.0,...,0,76ers,0,1,134,23:18,Covid,8000,0fca55cb-4bc8-4646-a2e6-75d95a20270b,2020-08-15T01:00:00+00:00
182366,8967,8967,Glenn,Robinson III,7f462af0-2ac8-4ca5-aa5a-17b37dc5001b,F,0,0.0,0.000,0.0,...,0,76ers,0,1,134,00:00,Covid,8000,0fca55cb-4bc8-4646-a2e6-75d95a20270b,2020-08-15T01:00:00+00:00
182367,8968,8968,Raul,Neto,8e7ffd66-f779-418c-bf18-b9f746a1c5fe,G,14,71.4,60.000,33.3,...,0,76ers,0,1,134,18:44,Covid,8000,0fca55cb-4bc8-4646-a2e6-75d95a20270b,2020-08-15T01:00:00+00:00


In [5]:
#dropping Crowd from df_dropped to check status weight v win & try to tweat training % up from 71
#added Crowd back in after initial run of 59% accuracy!
df_dropped = df[df['Min_played'] != "00:00"]
df_dropped = df_dropped[df_dropped['Crowd'] != 'Covid']
df_dropped = df_dropped[df_dropped['Crowd'] != '0']
df_dropped= df_dropped[["Points", "Free_Throw_Percent",
                  "Two_Pt_Percent",
                  "Three_Pt_Percent", "Assists",
                  "Rebounds", "Offensive_Rebounds",
                  "Steals", "Personal_Fouls",
                  "Flagrant_Fouls", "Tech_Fouls",
                  "Turnovers",
                  "Home_Away", "win"
                  ]].reset_index(drop = True)
df_dropped

,Points,Free_Throw_Percent,Two_Pt_Percent,Three_Pt_Percent,Assists,Rebounds,Offensive_Rebounds,Steals,Personal_Fouls,Flagrant_Fouls,Tech_Fouls,Turnovers,Home_Away,win
0,19,50.0,81.818,0.0,14,11,3,0,3,0,0,4,1,1
1,5,100.0,0.000,100.0,0,0,0,0,1,0,0,0,1,1
2,8,0.0,20.000,25.0,2,3,0,1,1,0,0,0,1,1
3,4,0.0,100.000,0.0,2,4,0,3,0,0,0,0,1,1
4,23,75.0,44.444,33.3,2,12,2,3,3,0,0,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120379,12,0.0,50.000,40.0,3,8,1,1,4,0,0,2,0,0
120380,9,100.0,100.000,66.7,0,2,0,0,3,0,0,0,0,0
120381,25,100.0,63.600,27.3,5,6,0,1,2,0,0,4,0,0
120382,14,0.0,87.500,0.0,8,13,2,0,3,0,0,5,0,0


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [7]:
X = df_dropped.drop('win', axis=1)
print(X.shape)

(120384, 13)


In [8]:
y = df_dropped['win']
print(y.shape)

(120384,)


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)

In [10]:
X_train.head()

,Points,Free_Throw_Percent,Two_Pt_Percent,Three_Pt_Percent,Assists,Rebounds,Offensive_Rebounds,Steals,Personal_Fouls,Flagrant_Fouls,Tech_Fouls,Turnovers,Home_Away
45899,18,0.0,0.000,66.7,1,2,0,2,0,0,0,2,1
38184,19,100.0,85.714,11.1,3,4,0,0,2,0,0,0,0
9181,5,0.0,33.333,20.0,2,1,0,1,2,0,0,2,0
59738,9,50.0,25.000,40.0,6,5,0,0,0,0,0,3,0
12545,12,0.0,60.000,0.0,1,16,9,0,3,0,0,3,1


In [11]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)


In [13]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

## Create a Deep Learning Model

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [28]:
# Create model and add layers
# tried increasing units by 100 per layer; less accurate
#best accuracy was three additional layers, 100 units, 100 epochs
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=13))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [29]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [30]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 100)               1400      
_________________________________________________________________
dense_10 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_11 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_12 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_13 (Dense)             (None, 2)                 202       
Total params: 31,902
Trainable params: 31,902
Non-trainable params: 0
_________________________________________________________________


In [31]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
2822/2822 - 1s - loss: 0.6714 - accuracy: 0.5888
Epoch 2/100
2822/2822 - 1s - loss: 0.6672 - accuracy: 0.5951
Epoch 3/100
2822/2822 - 1s - loss: 0.6661 - accuracy: 0.5964
Epoch 4/100
2822/2822 - 2s - loss: 0.6649 - accuracy: 0.5988
Epoch 5/100
2822/2822 - 2s - loss: 0.6645 - accuracy: 0.5997
Epoch 6/100
2822/2822 - 1s - loss: 0.6641 - accuracy: 0.5983
Epoch 7/100
2822/2822 - 1s - loss: 0.6635 - accuracy: 0.5994
Epoch 8/100
2822/2822 - 1s - loss: 0.6630 - accuracy: 0.6004
Epoch 9/100
2822/2822 - 1s - loss: 0.6624 - accuracy: 0.6004
Epoch 10/100
2822/2822 - 1s - loss: 0.6622 - accuracy: 0.6010
Epoch 11/100
2822/2822 - 1s - loss: 0.6617 - accuracy: 0.6016
Epoch 12/100
2822/2822 - 1s - loss: 0.6611 - accuracy: 0.6023
Epoch 13/100
2822/2822 - 1s - loss: 0.6605 - accuracy: 0.6028
Epoch 14/100
2822/2822 - 1s - loss: 0.6600 - accuracy: 0.6040
Epoch 15/100
2822/2822 - 1s - loss: 0.6595 - accuracy: 0.6043
Epoch 16/100
2822/2822 - 1s - loss: 0.6592 - accuracy: 0.6051
Epoch 17/100
2822

## Quantify our Trained Model

In [32]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

941/941 - 0s - loss: 0.8665 - accuracy: 0.6039
Normal Neural Network - Loss: 0.8664777874946594, Accuracy: 0.6039008498191833


In [33]:
encoded_predictions = model.predict_classes(X_test_scaled)
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [34]:
print(f"First 10 Predictions:   {prediction_labels[:10]}")
print(f"First 10 Actual labels: {y_test[:10].tolist()}")

First 10 Predictions:   [1 1 0 1 1 1 0 0 1 1]
First 10 Actual labels: [0, 1, 0, 0, 0, 1, 0, 1, 1, 1]


In [35]:
pd.DataFrame({"Prediction": prediction_labels, "Actual": y_test}).reset_index(drop=True)

,Prediction,Actual
0,1,0
1,1,1
2,0,0
3,1,0
4,1,0
...,...,...
30091,0,1
30092,1,1
30093,1,1
30094,0,0


In [50]:
model.save('models/deepLearningSequential.h5')